# 2-3-1. 파이썬으로 말뭉치 전처리하기

In [1]:
#말뭉치로 사용할 예시 문장
text = 'You say goodbye and I say hello.'

In [4]:
#문장 -> 단어 단위로 분할
text = text.lower()          #모든 문자를 소문자로 변환
text = text.replace('.', ' .')
print(text)

words = text.split(' ')      #공백을 기준으로 분할 #정규표현식을 사용하는 게 정석! 
                             #re.split('(\W+)?.text')를 호출하면 단어 단위로 분할할 수 있음.
print(words)

you say goodbye and i say hello  .
['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '', '.']


In [5]:
# 단어에 ID를 부여하고, ID의 리스트로 이용할 수 있도록 손질.
                   # Key = ID, Value = 단어
word_to_id = {}    #단어 -> ID로의 변환
id_to_word = {}    #ID -> 단어로의 변환

for word in words:    #단어 단위로 분할된 words의 각 원소를 처음부터 하나씩 살펴본다.
    if word not in word_to_id:   #word_to_id에 없는 단어는 word_to_id, id_to_word에 각각 새로운 id와 단어를 추가.
        new_id = len(word_to_id) #추가 시점의 딕셔너리 길이가 새로운 단어의 ID로 설정됨(0, 1, 2, ....)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [6]:
print(id_to_word)

print(word_to_id)

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '', 7: '.'}
{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '': 6, '.': 7}


In [11]:
#dict -> 단어 ID를 검색하거나, ID로 단어를 검색할 수 있다.
print(id_to_word[1])

print(word_to_id['and'])

say
3


In [12]:
#마지막으로 '단어 목록'을 '단어 ID 목록'으로 변환
import numpy as np

corpus = [word_to_id[w] for w in words] 
#comprehension(내포) 표기를 사용 -> 변환 '단어 목록'을 '단어 ID 목록'으로 변환 -> 다시 numpy 배열로 반환
#내포: 리스트, 딕셔너리 등의 반복문 처리를 간단하게 하는 기법.
#ex. xs = [1, 2, 3, 4]에서 각 원소를 제곱한 새로운 리스트를 생성하려면?
# [xs**2 for x in xs]
corpus = np.array(corpus)
print(corpus)

[0 1 2 3 4 1 5 6 7]


In [1]:
#이상의 전처리 과정을 함수화하기
import numpy as np

def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')
    
    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
        
    corpus = np.array([word_to_id[w] for w in words])
    
    return corpus, word_to_id, id_to_word

In [16]:
text = 'You say goodbye and I say hello'
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus, word_to_id, id_to_word)

[0 1 2 3 4 1 5] {'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5} {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello'}


# 2-3-4. 동시행렬 발생

#### 주목하는 단어 주변에 어떤 단어가 몇 번이나 등장하는가 집계하는 통계 기반 기법

In [18]:
#1단계: 위에서 사용한 말뭉치와 전처리 함수 사용 -> 전처리 진행

import sys
sys.path.append('..')
import numpy as np
from common.util import preprocess

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

print(corpus)

print(id_to_word)

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [19]:
#2단계: 각 단어의 맥락에 해당하는 단어의 빈도를 세어 보기
#window size = 1, ID = 0인 you 부터 시작.
#you의 맥락(1개): say
#맥락으로써 등장하는 단어의 빈도:
#      you   say   goodbye   and   i   hello   .
#you    0     1       0       0    0     0     0
#say    1     0       1       0    1     1     0
#.........
# .     0     0       0       0    0     1     0

#위와 같이 모든 단어에 동시발생하는 단어를 표에 정리한 것을 '동시발생 행렬(co-occurence matrix)'라고 한다.

In [22]:
#동시발생 행렬 구현(수동)
C = np.array([
    [0, 1, 0, 0, 0, 0, 0], #you
    [1, 0, 1, 0, 1, 1, 0], #say   
    [0, 1, 0, 1, 0, 0, 0], #goodbye
    [0, 0, 1, 0, 1, 0, 0], #and
    [0, 1, 0, 1, 0, 0, 0], #i
    [0, 1, 0, 0, 0, 0, 1], #hello
    [0, 0, 0, 0, 0, 1, 0]  #.
], dtype = np.int32)

In [23]:
print(C[0])   #ID = 0인 단어의 벡터 표현
print(C[4])
print(C[word_to_id['goodbye']])  #"goodbye"의 벡터 표현

[0 1 0 0 0 0 0]
[0 1 0 1 0 0 0]
[0 1 0 1 0 0 0]


In [31]:
#동시발생 행렬 구현(자동)
#함수로 작성할 수 있음.

#단어 ID의 리스트, 어휘 수, 윈도우 크기를 인수로 받음.
def create_co_matrix(corpus, vocab_size, window_size = 1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype = np.int32)  #먼저, co_matrix를 0으로 채워진 2차원 배열로 초기화.
    
    for idx, word_id in enumerate(corpus):    #말뭉치(corpus)의 모든 단어 각각에 대해 윈도우에 포함된 주변 단어를 세어나감.
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i
            
            if left_idx >= 0:                #corpus의 왼쪽 끝과 오른쪽 끝 경계를 벗어나지 않는지 확인.
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1
                
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
                
    return co_matrix

In [35]:
create_co_matrix(text, 7, window_size = 1)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

# 2-3-5. 벡터 간 유사도

In [29]:
#코사인 유사도를 파이썬 함수로 구현해 보자.
def cos_similarity(x, y):
    nx = x / np.sqrt(np.sum(x**2))  #x의 정규화 : 벡터 x의 각 원소를 제곱한 값을 다 더한 값에 제곱근 씌움. 
    ny = y / np.sqrt(np.sum(y**2))  #y의 정규화
    return np.dot(nx, ny)

#인수 x, y는 numpy 배열이라고 가정한다.
#각 벡터를 정규화한 후 두 벡터의 내적을 구하기.

#But, 문제가 있음: 인수로 제로 벡터(=원소가 모두 0인 벡터)가 들어오면 '0으로 나누기(divide by zero) 오류' 발생.

In [30]:
#'0으로 나누기 오류' 해결방법: 나눌 때 분모에 작은 값을 더해주는 것.

#1e-8은 거의 0에 가까운 아주 작은 값. 최종 계산에는 거의 영향을 주지 않되 오류는 막아준다.
def cos_similarity(x, y, eps = 1e-8):
    nx = x/np.sqrt(np.sum(x**2) + eps)
    ny = y/np.sqrt(np.sum(y**2) + eps)
    return np.dot(nx, ny)

In [6]:
#위 함수를 이용해 'you'와 'I'의 유사도 구하기
import numpy as np
import sys
sys.path.append('..')
from common.util import preprocess, create_co_matrix, cos_similarity

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

c0 = C[word_to_id['you']]
c1 = C[word_to_id['i']]
print(cos_similarity(c0, c1))  #0.7071... -1 <= 코사인 유사도 <= 1 이므로 꽤 높다.

0.7071067691154799


# 2-3-6. 유사 단어의 랭킹 표시

In [7]:
#어떤 단어가 검색어로 주어지면, 그 검색어와 비슷한 단어를 유사도 순으로 출력하는 함수를 만들자.

#most_similar함수의 5개 인자
## 1. query: 검색어(단어)
## 2. word_to_id: 단어에서 단어 ID로의 딕셔너리
## 3. id_to_word: 단어 ID에서 단어로의 딕셔너리
## 4. word_matrix: 단어 벡터들을 한 데 모은 행렬. 각 행에는 대응하는 단어의 벡터가 저장되어있다고 가정.
## 5. top: 상위 몇 개까지 출력할지 결정.

def most_similar(query, word_to_id, id_to_word, word_matrix, top = 5):
    #1)검색어를 꺼낸다.
    if query not in word_to_id:
        print("%s(을)를 찾을 수 없습니다."%query)
        return
    
    print('\n[query]' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    
    #2)코사인 유사도 계산.
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)
        
    #3)코사인 유사도를 기준으로 내림차순(값이 높은 순서대로)으로 출력
    #similarity 배열에 담긴 원소의 인덱스를 내림차순으로 정렬 후 상위 원소들을 출력
    count = 0
    for i in (-1 * similarity).argsort():  #argsort(): 배열 인덱스의 정렬 변경. 
                                           #numpy 배열의 원소를 오름차순으로 정렬하며, 
                                           #반환값은 배열의 인덱스.
        if id_to_word[i] == query:
            continue
        print(' %s: %s'%(id_to_word[i], similarity[i]))
        
        count += 1
        if count >= top:
            return                     

In [10]:
#argsort 예시
x = np.array([100, -20, 2])
print(x.argsort())    #[100, -20, 2] -> 오름차순 정렬[-20, 2, 100] -> 원래 인덱스 순서대로 출력하면 1, 2, 0
print((-x).argsort()) #x를 큰 순서대로 정렬하려면... 100, 2, -20의 역순으로 출력해야 한다. 
                      #따라서 x에 -를 곱해주면 내림차순으로 출력함.

[1 2 0]
[0 2 1]


In [11]:
#위에서 생성한 most_similar()함수 사용하여 'you' 검색하고 유사한 단어들 출력.

import sys
sys.path.append('..')
from common.util import preprocess, create_co_matrix, most_similar

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

most_similar('you', word_to_id, id_to_word, C, top=5)   # 단어: 코사인유사도


[query] you
 goodbye: 0.7071067691154799
 i: 0.7071067691154799
 hello: 0.7071067691154799
 say: 0.0
 and: 0.0
